LIMPIAR CSV

In [1]:
import sys
from pathlib import Path
import pandas as pd

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent/"src"
sys.path.append(str(path_inicio))
from utilidades.constantes import DATA_PATH, PATRON_ARCHIVOS_PATH, ARCHIVO_SALIDA_PATH, DATA_LIMPIA_PATH

In [12]:

# 1. Leer archivo consolidado
df = pd.read_csv(ARCHIVO_SALIDA_PATH, encoding='utf-8-sig')

# 2. Eliminar ventas anuladas (si existe la columna "Anulado")
if "Anulado" in df.columns:
    df = df[df["Anulado"].str.lower() == "no"]
else:
    print("No se encontró columna 'Anulado'.")

# 3. Estandarizar columna de fecha
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

# 4. Asegurar que TotalVenta y Cantidad sean numéricos
df['TotalVenta'] = pd.to_numeric(df['TotalVenta'].round(0).astype('Int64') , errors='coerce')
if 'Cantidad' in df.columns:
    df['Cantidad'] = pd.to_numeric(df['Cantidad'], errors='coerce')


columnas_utiles = ['Fecha', 'Producto', 'Cantidad', 'TotalVenta','Cond.Pago','Cliente']
df = df[columnas_utiles]

# 6. Guardar CSV limpio
df.to_csv(DATA_LIMPIA_PATH, index=False, encoding='utf-8-sig')
print("Archivo limpio guardado en data_procesada/ventas_limpio.csv")

# Verificá primeras filas
df.head()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12816\3829111835.py:11: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')


Archivo limpio guardado en data_procesada/ventas_limpio.csv


,Fecha,Producto,Cantidad,TotalVenta,Cond.Pago,Cliente
0,2025-06-30 14:07:10,ALMUERZO,1.0,15000,.CREDITO,EVEREX ARGENTINA SOCIEDAD ANONIMA
1,2025-06-30 10:07:59,MEDIALUNAS,24.0,21600,EFECTIVO,MOTOR PLAT SA
2,2025-06-27 14:18:18,ALMUERZO,1.0,15000,.CREDITO,EVEREX ARGENTINA SOCIEDAD ANONIMA
3,2025-06-27 11:05:54,SERVICIO DE CATERING,1.0,91600,.CREDITO,BALIARDA SOCIEDAD ANONIMA
4,2025-06-27 10:41:17,MEDIALUNAS,24.0,21600,.DEBITO,DIAGMED S A
